In [1]:
import numpy as np
import pandas as pd
import math
import itertools
import matplotlib.pyplot as plt
import sys
import torch  
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd


import import_ipynb
from Environments import Square_Crossroads

importing Jupyter notebook from Environments.ipynb


In [2]:
# n_times_steps = 3
# seed = 10
# dist_cars = 0.5

# env = Square_Crossroads(seed, dist_cars)
# env.reset()

# action_space = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0]}
# total_reward = 0

# print(env.states)


In [3]:
# state_tensor = torch.zeros([4,2], dtype=torch.float32)
# action_tensor = torch.zeros([4,2], dtype=torch.float32)

In [4]:
# state_tensor.shape[1]

In [7]:
class Actor_Critic(nn.Module):
    def __init__(self, num_inputs, hidden_size_actor, hidden_size_critic):
        super(Actor_Critic, self).__init__()
      
        self.num_inputs = num_inputs
        
        
        self.critic_linear1 = nn.Linear(self.num_inputs, hidden_size_critic)
        self.critic_linear2 = nn.Linear(hidden_size_critic, 1)
        
        self.actor_linear1 = nn.Linear(self.num_inputs, hidden_size_actor)
        self.actor_linear2 = nn.Linear(hidden_size_actor, hidden_size_actor)
        self.alpha = nn.Linear(hidden_size_actor, 1)
        self.beta = nn.Linear(hidden_size_actor, 1)
        
        self.pos_neg_1 = nn.Linear(self.num_inputs, 10)
        self.pos_neg_2 = nn.Linear(10, 2)

    def forward(self, state_tensor):
#         print(state_tensor)
        state_tensor = Variable(state_tensor.float())
#         print(state_tensor)
        value = F.relu(self.critic_linear1(state_tensor))
        value = self.critic_linear2(value)
        
        policy_dist = F.relu(self.actor_linear1(state_tensor))
        policy_dist = F.relu(self.actor_linear2(policy_dist))

        policy_dist_alpha = torch.clamp(self.alpha(policy_dist), 1.9, 2.2)
        policy_dist_beta = torch.clamp(self.beta(policy_dist), 1.9, 2.2)
        
        sign = F.relu(self.pos_neg_1(state_tensor))
        sign = F.softmax(self.pos_neg_2(sign))
        
        return value, policy_dist_alpha, policy_dist_beta, sign